In [1]:
import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

from keras.preprocessing.image import ImageDataGenerator

In [4]:
IMAGE_SIZE = 256
CHANNELS = 3
train_datagen = ImageDataGenerator(
    rescale=1./255, # scaling by deviding by 255
    horizontal_flip=True,
    rotation_range=10, # by 10 degree rotate the image
        
)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse',
    # save_to_dir="AugmentedImages"
    
)

Found 2896 images belonging to 58 classes.


In [5]:
for image_batch, label_batch in train_generator:
    print(image_batch[0])
    break

[[[0.9176471  0.9163888  0.9333334 ]
  [0.9176471  0.91663694 0.9333334 ]
  [0.9176471  0.91688496 0.9333334 ]
  ...
  [0.9450981  0.9450981  0.9607844 ]
  [0.9450981  0.9450981  0.9607844 ]
  [0.9424348  0.9450981  0.95812106]]

 [[0.9163967  0.914976   0.9333334 ]
  [0.91664475 0.9147279  0.9333334 ]
  [0.9168928  0.9144798  0.9333334 ]
  ...
  [0.9450981  0.9450981  0.9607844 ]
  [0.9450981  0.9450981  0.9607844 ]
  [0.9421867  0.9450981  0.957873  ]]

 [[0.91372555 0.9176471  0.9333334 ]
  [0.91372555 0.9176471  0.9333334 ]
  [0.91372555 0.9176471  0.9333334 ]
  ...
  [0.9450981  0.9450981  0.9607844 ]
  [0.9450981  0.9450981  0.9607844 ]
  [0.9419387  0.9450981  0.957625  ]]

 ...

 [[0.6112243  0.50675535 0.46285596]
  [0.5764706  0.49411768 0.44705886]
  [0.57497776 0.484414   0.4373552 ]
  ...
  [0.9215687  0.9215687  0.9215687 ]
  [0.9215687  0.9215687  0.9215687 ]
  [0.9215687  0.9215687  0.9215687 ]]

 [[0.6084955  0.50576305 0.46161562]
  [0.5764706  0.49411768 0.44705886]


In [6]:
validation_datagen = ImageDataGenerator(
    rescale=1./255, # scaling by deviding by 255
    horizontal_flip=True,
    rotation_range=10, # by 10 degree rotate the image
        
)

validation_generator = validation_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse', # 0 1 2 ..
    
)

Found 394 images belonging to 58 classes.


In [7]:
test_datagen = ImageDataGenerator(
    rescale=1./255, # scaling by deviding by 255
    horizontal_flip=True,
    rotation_range=10, # by 10 degree rotate the image
        
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=32,
    class_mode='sparse', # 0 1 2 ..
    
)

Found 880 images belonging to 58 classes.


In [8]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 58

model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),

])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)        0

In [10]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [12]:
# number of training images devided by the number of images in one batch gives us how many batches we need.
2896/32

90.5

In [13]:
394/32

12.3125

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=90,
    batch_size=32,
    validation_data=validation_generator,
    validation_steps=12,
    verbose=1,
    epochs=20,
)

In [ ]:
scores = model.evaluate(test_generator)

In [ ]:
scores

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
type(history.history['loss'])

In [ ]:
len(history.history['loss'])

In [ ]:
history.history['loss'][:5] # show loss for first 5 epochs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
val_acc

In [ ]:
acc

In [ ]:
EPOCHS = 20

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

In [ ]:
import numpy as np

for image_batch, label_batch in test_generator:
    first_image = image_batch[0]
    first_label = int(label_batch[0])
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(image_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])
    
    break

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_generator:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_names[int(labels[i])] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")
    break

In [ ]:
model.save("../potatoes.h5")